# Calculator On time percent 

In [1]:
import yaml
import pandas as pd
import numpy as np
from lxml import etree

import os

# Set up event and vehicle types

In [2]:
TRANSIT_VEHICLE_PATH = "data\\simple_scenario\\transitVehicles.xml"
EVENT_PATH = "data\\simple_scenario\\output\\output_events.xml"

OUTPUT_EVENT_AFTER_PROCESSOR = "data\\simple_scenario\\scoring\\otp\\bus_delay_at_stop.csv"

# TRANSIT_VEHICLE_PATH = "data\\real\\transitVehicles.xml"
# EVENT_PATH = "data\\real\\output_events_Hai.xml"

# OUTPUT_EVENT_AFTER_PROCESSOR = "data\\real\\scoring\\otp\\bus_delay_at_stop.csv"


if os.path.exists(OUTPUT_EVENT_AFTER_PROCESSOR):
    os.remove(OUTPUT_EVENT_AFTER_PROCESSOR)

folder_path = os.path.dirname(OUTPUT_EVENT_AFTER_PROCESSOR)
os.makedirs(folder_path, exist_ok=True)

# Xử lý file transit vehicle

In [3]:
!powershell -Command "Get-Content 'data\\simple_scenario\\transitVehicles.xml' -TotalCount 20 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"

5: <?xml version="1.0" encoding="UTF-8"?>
5: <vehicleDefinitions xmlns="http://www.matsim.org/files/dtd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.matsim.org/files/dtd http://www.matsim.org/files/dtd/vehicleDefinitions_v1.0.xsd">
5:   <vehicleType id="bus_type"> <capacity><seats persons="50"/><standingRoom persons="0"/></capacity> <length meter="10.0"/> </vehicleType>
5:   <vehicleType id="tram_type"> <capacity><seats persons="50"/><standingRoom persons="0"/></capacity> <length meter="10.0"/> </vehicleType>
5:   <vehicleType id="train_type"> <capacity><seats persons="100"/><standingRoom persons="0"/></capacity> <length meter="10.0"/> </vehicleType>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_0" type="bus_type"/>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_1" type="bus_type"/>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_2" type="bus_type"/>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_3" type="bus_type"/>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_4" type="bus_typ

### Xử lý ra dict chứa id và type

In [4]:
tree = etree.parse(TRANSIT_VEHICLE_PATH)
root = tree.getroot()

#### xlmn trong thẻ vehicleDefinitions  là namespace cho tất cả các tag trong tag này. nghĩa là tên đầy đủ của tag là {http://www.matsim.org/files/dtd}vehicleDefinitions, {http://www.matsim.org/files/dtd}vehicle.

#### Khi dùng lxml phải dùng namespace: root.xpath("//m:vehicleDefinitions/m:vehicle", namespaces=ns)

In [5]:
ns = {'m': 'http://www.matsim.org/files/dtd'}
vehtype_dict = {}


for node in root.xpath("//m:vehicleDefinitions/m:vehicle", namespaces=ns):
    id = node.xpath("@id")[0]
    type = node.xpath("@type")[0]
    print([id,type])
    vehtype_dict[id] = type



['veh_Bus_Line_1_dir_fwd_0', 'bus_type']
['veh_Bus_Line_1_dir_fwd_1', 'bus_type']
['veh_Bus_Line_1_dir_fwd_2', 'bus_type']
['veh_Bus_Line_1_dir_fwd_3', 'bus_type']
['veh_Bus_Line_1_dir_fwd_4', 'bus_type']
['veh_Bus_Line_1_dir_bwd_0', 'bus_type']
['veh_Bus_Line_1_dir_bwd_1', 'bus_type']
['veh_Bus_Line_1_dir_bwd_2', 'bus_type']
['veh_Bus_Line_1_dir_bwd_3', 'bus_type']
['veh_Bus_Line_1_dir_bwd_4', 'bus_type']
['veh_Bus_Line_2_dir_fwd_0', 'bus_type']
['veh_Bus_Line_2_dir_fwd_1', 'bus_type']
['veh_Bus_Line_2_dir_fwd_2', 'bus_type']
['veh_Bus_Line_2_dir_fwd_3', 'bus_type']
['veh_Bus_Line_2_dir_fwd_4', 'bus_type']
['veh_Bus_Line_2_dir_bwd_0', 'bus_type']
['veh_Bus_Line_2_dir_bwd_1', 'bus_type']
['veh_Bus_Line_2_dir_bwd_2', 'bus_type']
['veh_Bus_Line_2_dir_bwd_3', 'bus_type']
['veh_Bus_Line_2_dir_bwd_4', 'bus_type']
['veh_Bus_Line_3_dir_fwd_0', 'bus_type']
['veh_Bus_Line_3_dir_fwd_1', 'bus_type']
['veh_Bus_Line_3_dir_fwd_2', 'bus_type']
['veh_Bus_Line_3_dir_fwd_3', 'bus_type']
['veh_Bus_Line_3

## Xử lý event

In [6]:
!powershell -Command "Get-Content 'data\\simple_scenario\\output\\output_events.xml' -TotalCount 100 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"

5: <?xml version="1.0" encoding="utf-8"?>
5: <events version="1.0">
5: 	<event time="21600.0" type="TransitDriverStarts" driverId="pt_veh_Tram_Line_1_dir_fwd_0_tram_type" vehicleId="veh_Tram_Line_1_dir_fwd_0" transitLineId="Tram_Line_1" transitRouteId="dir_fwd" departureId="veh_Tram_Line_1_dir_fwd_0"  />
5: 	<event time="21600.0" type="departure" person="pt_veh_Tram_Line_1_dir_fwd_0_tram_type" link="7" legMode="car"  />
5: 	<event time="21600.0" type="PersonEntersVehicle" person="pt_veh_Tram_Line_1_dir_fwd_0_tram_type" vehicle="veh_Tram_Line_1_dir_fwd_0"  />
5: 	<event time="21600.0" type="TransitDriverStarts" driverId="pt_veh_Tram_Line_1_dir_bwd_0_tram_type" vehicleId="veh_Tram_Line_1_dir_bwd_0" transitLineId="Tram_Line_1" transitRouteId="dir_bwd" departureId="veh_Tram_Line_1_dir_bwd_0"  />
5: 	<event time="21600.0" type="departure" person="pt_veh_Tram_Line_1_dir_bwd_0_tram_type" link="74" legMode="car"  />
5: 	<event time="21600.0" type="PersonEntersVehicle" person="pt_veh_Tram_Line_

#### Chỉ đếm những người lên xe bus khác lái xe

In [7]:
hint_bus_type = "bus"
schema = ['vehicleId', 'vehicleType', 'facility' ,  'arrDelay', 'depDelay', 'arrTime', 'depTime']
temp_bus_map = {}

with open(OUTPUT_EVENT_AFTER_PROCESSOR, 'a') as f:
    for x in schema[:-1]:
        f.write(x + ",")
    f.write(schema[-1] + "\n")

context = etree.iterparse(EVENT_PATH, events=('end',))
for event, elem in context:
    if elem.tag == "event":
        e_type = elem.get("type")

        if e_type == "VehicleArrivesAtFacility":

            veh_id = elem.get("vehicle")
            delay = elem.get("delay")
            time = elem.get("time")
            

            if hint_bus_type not in vehtype_dict[veh_id].lower():
                continue           
            if delay is None:
                delay = "null"    

            veh_type = vehtype_dict[veh_id]


            temp_bus_map[veh_id] = {
                "vehType": veh_type,
                "arrDelay": float(delay),
                "depDelay": 0.0,
                "arrTime": float(time),
                "depTime": 0.0
            }

        elif e_type == "VehicleDepartsAtFacility":

            veh_id = elem.get("vehicle")
            delay = elem.get("delay")
            time = elem.get("time")
            facility = elem.get("facility")

            if veh_id not in temp_bus_map:
                continue
            if delay is None:
                delay = "0.0"
            
            temp_bus_map[veh_id]["depTime"] = float(time)
            temp_bus_map[veh_id]["depDelay"] = float(delay)

            with open(OUTPUT_EVENT_AFTER_PROCESSOR, 'a') as f:
                f.write(f"{veh_id}, {temp_bus_map[veh_id]['vehType']}, {facility}, {temp_bus_map[veh_id]['arrDelay']}, {temp_bus_map[veh_id]['arrTime']}, {temp_bus_map[veh_id]['depDelay']} ,{temp_bus_map[veh_id]['depTime']}\n")

            del temp_bus_map[veh_id]

            
    elem.clear()

print("Finished processing otp data.")





Finished processing otp data.


# Tính OTP

In [8]:
def calculte_otp(bus_delay_path: str, max_delay: float = 180.0, min_delay: float = -180.0):
    
    delaydf = pd.read_csv(bus_delay_path)
    ontime = delaydf[(delaydf["arrDelay"] <= max_delay) & (delaydf["arrDelay"] >= min_delay)].shape[0]
    total = delaydf.shape[0]
    otp_percent = ontime / total * 100.0

    return ontime, total, otp_percent

ontime, total, otp_percent = calculte_otp(OUTPUT_EVENT_AFTER_PROCESSOR)
print(f"On-time arrivals: {ontime} / {total}  --> OTP: {otp_percent:.2f} %")



On-time arrivals: 60 / 150  --> OTP: 40.00 %
